<a href="https://colab.research.google.com/github/superbunny38/2021DeepLearning/blob/main/pytorch/Chap_8_(3)_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from glob import glob
import os
from shutil import copyfile
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18,resnet34
from torchvision.models.inception import inception_v3
from torchvision.models import densenet121
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

%matplotlib inline

In [2]:
is_cuda = torch.cuda.is_available()
print(is_cuda)
print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()

True
0


In [24]:
def imshow(inp,cmap=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp,cmap)
    
class FeaturesDataset(Dataset):
    
    def __init__(self,featlst,labellst):
        self.featlst = featlst
        self.labellst = labellst
        
    def __getitem__(self,index):
        return (self.featlst[index],self.labellst[index])
    
    def __len__(self):
        return len(self.labellst)
    
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        
        running_loss += F.cross_entropy(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy}')
    return loss,accuracy

In [4]:
data_transform = transforms.Compose([
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [5]:
!git clone https://github.com/ardamavi/Dog-Cat-Classifier repo
!mkdir workspace
%mv /content/repo/Data/Train_Data /content/workspace
%cd workspace

Cloning into 'repo'...
remote: Enumerating objects: 1654, done.
remote: Total 1654 (delta 0), reused 0 (delta 0), pack-reused 1654
Receiving objects: 100% (1654/1654), 34.83 MiB | 14.19 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/workspace


In [7]:
import glob
path = "/content/workspace/Train_Data"
#read all the files inside the folder
dog_files = [f for f in glob.glob("/content/workspace/Train_Data/dog/*.jpg")]
cat_files = [f for f in glob.glob("/content/workspace/Train_Data/cat/*.jpg")]

files = dog_files+cat_files
print(f"Total number of images: {len(files)}")
no_of_images = len(files)
shuffle = np.random.permutation(no_of_images)
os.mkdir(os.path.join(path, "train"))
os.mkdir(os.path.join(path, "valid"))
for t in ['train','valid']:
  for folder in ['dog/', 'cat/']:
    os.mkdir(os.path.join(path,t,folder))

Total number of images: 1399


In [8]:
#검증데이터 넣기
for i in shuffle[:250]:#250개
  folder = files[i].split("/")[-2].split(".")[0]
  image = files[i].split('/')[-1]
  image = image[4:]
  os.rename(files[i], os.path.join(path, 'valid', folder, image))

  #학습데이터 넣기
for i in shuffle[250:]:
  folder = files[i].split("/")[-2].split(".")[0]
  image = files[i].split('/')[-1]
  image = image[4:]
  os.rename(files[i], os.path.join(path, 'train', folder, image))

In [12]:
from torchvision.datasets import ImageFolder
train_dset = ImageFolder("/content/workspace/Train_Data/train", data_transform)
val_dset = ImageFolder("/content/workspace/Train_Data/valid", data_transform)
classes=2

In [13]:
train_loader = DataLoader(train_dset,batch_size=32,shuffle=False,num_workers=3)
val_loader = DataLoader(val_dset,batch_size=32,shuffle=False,num_workers=3)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Creating Densenet 121 model

In [14]:
my_densenet = densenet121(pretrained=True).features
if is_cuda:
    my_densenet = my_densenet.cuda()

for p in my_densenet.parameters():
    p.requires_grad = False

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

# Extracting Convolutional features

In [15]:
#For training data
trn_labels = []
trn_features = []

#code to store densenet features for train dataset.
for d,la in train_loader:
    with torch.no_grad(): # To help with memory issues
        o = my_densenet(Variable(d.cuda()))
        o = o.view(o.size(0),-1)
        trn_labels.extend(la.cpu())
        trn_features.extend(o.cpu().data)

#For validation data
val_labels = []
val_features = []

#Code to store densenet features for validation dataset. 
for d,la in val_loader:
    with torch.no_grad(): # To help with memory issues
        o = my_densenet(Variable(d.cuda()))
        o = o.view(o.size(0),-1)
        val_labels.extend(la.cpu())
        val_features.extend(o.cpu().data)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Creating train and validation feature dataset


In [16]:
# Create dataset for train and validation convolution features
trn_feat_dset = FeaturesDataset(trn_features,trn_labels)
val_feat_dset = FeaturesDataset(val_features,val_labels)

# Create data loaders for batching the train and validation datasets
train_feat_loader = DataLoader(trn_feat_dset,batch_size=64,shuffle=True,drop_last=True)
validation_feat_loader = DataLoader(val_feat_dset,batch_size=64)

# A Fully connected network

In [17]:
class FullyConnectedModel(nn.Module):
    
    def __init__(self,input_size,output_size):
        super().__init__()
        self.fc = nn.Linear(input_size,output_size)

    def forward(self,input):
        output = self.fc(input)
        return output

In [18]:
trn_features[0].size(0)

82944

In [19]:
fc_in_size = trn_features[0].size(0)

In [20]:
fc = FullyConnectedModel(fc_in_size,classes)
if is_cuda:
    fc = fc.cuda()

In [21]:
optimizer = optim.Adam(fc.parameters(),lr=0.0001)

# Train

In [25]:
train_losses , train_accuracy = [],[]
validation_losses , validation_accuracy = [],[]
for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch,fc,train_feat_loader,phase='training')
    validation_epoch_loss , validation_epoch_accuracy = fit(epoch,fc,validation_feat_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    validation_losses.append(validation_epoch_loss)
    validation_accuracy.append(validation_epoch_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training loss is 0.33469271659851074 and training accuracy is 908/114979.02523803710938
validation loss is 0.18179941177368164 and validation accuracy is 229/25091.5999984741211
training loss is 0.029531167820096016 and training accuracy is 1085/114994.42993927001953
validation loss is 0.1839035600423813 and validation accuracy is 235/25094.0
training loss is 0.009118863381445408 and training accuracy is 1088/114994.69103240966797
validation loss is 0.15473072230815887 and validation accuracy is 237/25094.80000305175781
training loss is 0.005111027508974075 and training accuracy is 1088/114994.69103240966797
validation loss is 0.16076406836509705 and validation accuracy is 238/25095.19999694824219
training loss is 0.003816070733591914 and training accuracy is 1088/114994.69103240966797
validation loss is 0.15830475091934204 and validation accuracy is 237/25094.80000305175781
training loss is 0.0030860668048262596 and training accuracy is 1088/114994.69103240966797
validation loss is 0.